##### Copyright 2023 Google LLC

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

### Install & import

In [1]:
!pip install -U -q google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.2/164.2 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.3/718.3 kB 7.6 MB/s eta 0:00:00


In [2]:
!pip install -U -q PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.0 MB/s eta 0:00:00


In [3]:
# Install the client library and import necessary modules.
import google.generativeai as genai

import os
from typing import Dict
from google.colab import drive
from io import BytesIO
import PyPDF2

import base64
import io
import json
import mimetypes
import pathlib
import pprint
import requests
import pandas as pd
import numpy as np

import PIL.Image
import IPython.display
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import Markdown

In [4]:
from google.colab import auth
auth.authenticate_user()

In [5]:
from googleapiclient.discovery import build

drive_service = build('drive', 'v3')
files = drive_service.files().list().execute()

In [6]:
from google.colab import userdata

API_KEY=userdata.get('key090524')

In [7]:
# Configure the client library by providing your API key.
genai.configure(api_key=API_KEY)

In [8]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive




---
⏰# **LONG PREP STEP STARTS HERE**: ⏰


The next cells will perform a code that takes some time. It will read each pdf, capture the info in the text and split the text into smaller chunks increasing the length of the dataframe. After this there will be also a step to embbed each of the phares, this is so far the longest step in the process. If you have already used the code you will have the whole dataframe with embeddings saved as csv (or any other format of your preference) so it is better to just load it from there.

In [ ]:
def pdf_to_text_dict(folder_path: str) -> Dict[str, str]:
  """
  Parses all PDF files within a Google Drive folder and returns a dictionary.

  Args:
      folder_path: Path to the folder containing the PDF files within Google Drive.

  Returns:
      A dictionary where the key is the PDF file name and the value is the extracted text.
  """

  pdf_text_dict = {}

  for filename in os.listdir(folder_path):
    if filename.endswith(".pdf"):
      filepath = os.path.join(folder_path, filename)

      with open(filepath, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)

        num_pages = len(pdf_reader.pages)
        file_content = ""

        for page_num in range(num_pages):
          page = pdf_reader.pages[page_num]
          file_content += page.extract_text()

      pdf_text_dict[filename] = file_content

  return pdf_text_dict

# Replace with the actual path to your Google Drive folder
folder_path = '/gdrive/MyDrive/Colab Notebooks/bulas'

pdf_dict = pdf_to_text_dict(folder_path)

In [ ]:
old_keys = list(pdf_dict.keys())
new_keys = ['paracetamol','acido acetilsalicilico','metamizol','diclofenaco','amoxicilina']
for k in range(len(old_keys)):
  pdf_dict[new_keys[k]] = pdf_dict[old_keys[k]]
  del pdf_dict[old_keys[k]]

In [ ]:
def clean_text(text):
  """Removes extra lines, line breaks, and spaces from text."""
  text = text.replace('\n', ' ')  # Replace line breaks with spaces
  text = ' '.join(text.split())   # Remove extra spaces
  return text

pdf_dict_cleaned = {filename: clean_text(text) for filename, text in pdf_dict.items()}

In [ ]:
df_bulas = pd.DataFrame.from_dict(pdf_dict_cleaned, orient = 'index', columns = ['texto'])
df_bulas = df_bulas.reset_index()
df_bulas = df_bulas.rename(columns = {'index':'medicamento'})
df_bulas.head()

,medicamento,texto
0,paracetamol,"Indicações Em adultos, para a redução da febre..."
1,acido acetilsalicilico,anti-inflamatórios/anti-reumáticos e a outros ...
2,metamizol,dipirona monoidratada Brainfarma Indústria Quí...
3,diclofenaco,Blau Farmacêutica S/A. DICLOFENACO SÓDICO Blau...
4,amoxicilina,amoxicilina tri-hidratada Bula para profission...


In [ ]:
import re

# Function to split text into phrases
def split_into_phrases(text):
    return re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)

# Function to generate a key name from a phrase
def generate_key_name(phrase, max_words=5):
    words = phrase.split()
    return ' '.join(words[:max_words])

# List to store new rows
new_rows = []

# Process each row in the DataFrame
for idx, row in df_bulas.iterrows():
    phrases = split_into_phrases(row['texto'])
    for phrase in phrases:
        if phrase:  # Ensure phrase is not empty
            key_name = generate_key_name(phrase)
            new_rows.append({'medicamento': row['medicamento'] + '_' + key_name, 'texto': phrase})

# Create a new DataFrame with the new rows
df_bulas_split = pd.DataFrame(new_rows)

In [ ]:
df_bulas_split.head(20)

,medicamento,texto
0,"paracetamol_Indicações Em adultos, para a","Indicações Em adultos, para a redução da febre..."
1,paracetamol_Contra-indicações O paracetamol nã...,Contra-indicações O paracetamol não deve ser a...
2,paracetamol_Advertências Não use outro medicam...,Advertências Não use outro medicamento que con...
3,paracetamol_Precauções Embora o paracetamol possa,Precauções Embora o paracetamol possa ser util...
4,paracetamol_A administração deve ser feita,A administração deve ser feita por períodos cu...
5,paracetamol_O paracetamol não deve ser,O paracetamol não deve ser administrado por ma...
6,paracetamol_Usuários crônicos de bebidas alcoó...,Usuários crônicos de bebidas alcoólicas podem ...
7,paracetamol_O paracetamol pode causar dano,O paracetamol pode causar dano hepático.
8,paracetamol_Reações adversas Podem ocorrer alg...,Reações adversas Podem ocorrer algumas reações...
9,paracetamol_Caso ocorra uma rara reação,"Caso ocorra uma rara reação de sensibilidade, ..."


In [ ]:
model = 'models/embedding-001'

In [ ]:
def embedding_create(title, text):
  return genai.embed_content(model = model,content = text, title = title, task_type ='RETRIEVAL_DOCUMENT')['embedding']

In [ ]:
from google.api_core.exceptions import BadRequest
import google.generativeai as palm

def embed_text_with_title(text, title, chunk_size=500):
  """Embeds text in chunks, incorporating the title."""
  text = f"{title}. {text}" # Prepend title to the text
  words = text.split()
  embeddings = []
  for i in range(0, len(words), chunk_size):
    chunk = ' '.join(words[i:i + chunk_size])
    try:
      response = genai.embed_content(model="models/embedding-001", content=chunk, task_type ='RETRIEVAL_DOCUMENT')
      embeddings.append(response['embedding'])
    except BadRequest as e:
      print(f"Error embedding chunk: {e}")
      return None

  embeddings = [emb for emb in embeddings]
  #embeddings = embeddings.flatten()
  # Calculate max_length based on the length of the embeddings list
  max_length = max([len(emb) for emb in embeddings])
  # Pad the embeddings with zeros to the same length
  padded_embeddings = [np.pad(emb, (0, max_length - len(emb)), 'constant') for emb in embeddings]

  #new_embeddings = []
  #for emb in padded_embeddings:
  #  for i in emb:
  #    new_embeddings.append(i)


  return padded_embeddings


# **After running the bellow step for the first time the next code box will save it as CSV. Skip the bellow cell if you have already ran it and have the CSV. The bellow cell takes me up to 20 min to conclude running.**

In [ ]:
df_bulas_split['embeddings'] = df_bulas_split.apply(lambda row: embed_text_with_title(row['texto'], row['medicamento']), axis=1)
df_bulas_split.head()

NameError: name 'df_bulas_split' is not defined

In [ ]:
df_bulas_split.to_csv('/gdrive/MyDrive/Colab Notebooks/bulas/df_bulas_split.csv', index=False)

**If you already ran the code above you can continue from here and read the csv file directly to save time**



---
✅ # **LONG PREP STEP ENDS HERE** ✅



In [9]:
#/gdrive/MyDrive/Colab Notebooks/PJT chatbot alura.ipynb
df_bulas_split = pd.read_csv('/gdrive/MyDrive/Colab Notebooks/bulas/df_bulas_split.csv')

Bellow is a test to find a medicine with the normal string finding from pandas

In [10]:
df_bulas_split[df_bulas_split['medicamento'].str.contains('acetil',case=False, na=False)]

,medicamento,texto,embeddings
28,paracetamol_A acetilcisteína a 20 %,A acetilcisteína a 20 % deve ser administrada ...,"[array([ 2.87145230e-02, -3.59287750e-02, -6.8..."
29,"paracetamol_A acetilcisteína a 20%, deve","A acetilcisteína a 20%, deve ser dada após dil...","[array([ 1.98771400e-02, -3.66042330e-02, -6.5..."
30,paracetamol_Além da administração da acetilcis...,"Além da administração da acetilcisteína a 20%,...","[array([ 4.22642500e-02, -2.51915730e-02, -6.6..."
252,acido acetilsalicilico_anti-inflamatórios/anti...,anti-inflamatórios/anti-reumáticos e a outros ...,"[array([ 2.54086200e-03, -4.10816800e-02, -5.5..."
253,acido acetilsalicilico_derivados cumarínicos o...,derivados cumarínicos ou heparina – exceto ter...,"[array([ 0.02998589, -0.04108284, -0.06465814,..."
...,...,...,...
663,acido acetilsalicilico_Precauções e advertênci...,Precauções e advertências: Somente após rigoro...,"[array([ 0.00122238, -0.06297091, -0.05041644,..."
664,acido acetilsalicilico_Embalagem _____________...,Embalagem ___________________________ _____/__...,"[array([ 0.01366628, -0.02074629, -0.05128136,..."
665,acido acetilsalicilico_Marketing _____________...,Marketing ______________________________ _____...,"[array([ 1.25781390e-02, -3.53270060e-02, -4.4..."
666,acido acetilsalicilico_Galênico_______________...,Galênico_____________________________ _____/__...,"[array([ 1.98976730e-02, -5.08812550e-02, -7.2..."


In [11]:
import ast
#As from CSV everything was converted to string some changes need to be done in the dataframe
df_bulas_split['embeddings'] = df_bulas_split['embeddings'].str.replace('\n', '')
df_bulas_split['embeddings'] = df_bulas_split['embeddings'].str.replace('[array([', '')
df_bulas_split['embeddings'] = df_bulas_split['embeddings'].str.replace('])', '')
df_bulas_split['embeddings'] = df_bulas_split['embeddings'].str.replace('[', '')

In [12]:
df_bulas_split['embeddings'][1]

' 2.47939000e-02, -3.63911800e-02, -7.94218100e-02,  9.13854000e-03,        1.01060930e-01,  5.40322530e-03, -1.73252770e-02, -1.41867640e-02,       -1.91202900e-02,  6.16487000e-02, -2.17315870e-02,  1.28175090e-02,       -8.60075200e-04,  6.81002700e-04, -2.56608840e-02, -5.45868280e-02,       -1.12370600e-03,  1.66438180e-02, -4.76897370e-03, -2.66084100e-02,        8.93574800e-03,  2.19835020e-02, -1.69324630e-02,  1.73874940e-02,        1.74775700e-03, -2.21421510e-02, -2.92698420e-05, -6.56498900e-02,       -2.20014550e-02, -3.24690090e-03, -2.55385940e-02,  7.09511700e-02,       -4.30718440e-02,  6.62153170e-03, -2.19775850e-02, -3.39481050e-02,       -1.59400100e-02,  1.12673080e-02, -4.25842100e-02,  2.21631970e-02,        2.88319480e-02, -5.80330000e-03, -2.76516080e-02,  3.58850960e-02,       -2.51906100e-02,  1.21689040e-02, -5.58341780e-02,  4.31890650e-03,       -3.67837330e-03, -6.95855700e-02,  8.65179800e-03,  2.13448820e-02,        8.03641160e-02, -4.38472440e-02,  8.

In [19]:
copy = df_bulas_split['embeddings'].str.split(',',expand = True)
copy.columns

RangeIndex(start=0, stop=768, step=1)

In [20]:
for c in range(0,768):
  copy[c] = copy[c].str.replace(']','')
  copy[c] = copy[c].astype(float)

In [21]:
copy.values.to_list()

From csv it is converting to text when reading so this needs to be fixed further the line so the code works. Should be list format or something like that

In [ ]:
from google.api_core.exceptions import BadRequest
import google.generativeai as palm
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import ast

def embed_text_with_title(text, title, chunk_size=500):
  """Embeds text in chunks, incorporating the title."""
  text = f"{title}. {text}" # Prepend title to the text
  words = text.split()
  embeddings = []
  for i in range(0, len(words), chunk_size):
    chunk = ' '.join(words[i:i + chunk_size])
    try:
      response = palm.embed_content(model="models/embedding-001", content=chunk, task_type ='RETRIEVAL_DOCUMENT')
      embeddings.append(response['embedding'])
    except BadRequest as e:
      print(f"Error embedding chunk: {e}")
      return None

  # Flatten the list of embeddings into a single 1D array
  embeddings = np.concatenate(embeddings)
  return embeddings

def get_relevant_phrases(prompt, df_bulas):
  """
  Retrieves the most relevant phrases from the "texto" column based on a prompt.

  Args:
      prompt: A string containing the prompt.
      df_bulas: A pandas DataFrame containing the "medicamento" and "texto" columns.

  Returns:
      A list of the most relevant phrases.
  """

  # Extract the medicamento name from the prompt
  medicamento = prompt.split()[0]

  # Get the embeddings for the prompt and the medicamento text
  prompt_embeddings = embed_text_with_title(prompt, medicamento).reshape(1, -1) # Reshape to 2D
  #print(prompt_embeddings)
  # Extract embeddings and handle potential list of lists
  medicamento_embeddings = df_bulas[df_bulas['medicamento'].str.contains(medicamento, case = False, na = False)]['embeddings'].values
  #medicamento_embeddings = np.array([ast.literal_eval(emb) for emb in medicamento_embeddings])

  if isinstance(medicamento_embeddings[0], list):
    medicamento_embeddings = np.array([np.concatenate(emb) for emb in medicamento_embeddings])
  else:
    medicamento_embeddings = np.stack(medicamento_embeddings, axis=0) # Stack embeddings into 2D array

  # Calculate the cosine similarity between the prompt and each phrase in the medicamento text

  similarities = cosine_similarity(prompt_embeddings, medicamento_embeddings)

  # Sort the phrases by their similarity scores and return the top 5
  sorted_indices = np.argsort(similarities.flatten())[::-1][:5]

  relevant_phrases = []
  for index in sorted_indices:
    relevant_phrases.append(df_bulas.iloc[index]['texto'])

  return relevant_phrases

In [ ]:
# Example usage
prompt = "paracetamol adverso"
relevant_phrases = get_relevant_phrases(prompt, df_bulas_split)

# Print the most relevant phrases
for phrase in relevant_phrases:
  print(phrase)

ValueError: could not convert string to float: '[array([ 9.45394800e-03, -4.64319800e-02, -7.72205800e-02,  1.57492440e-02,        9.76961100e-02,  2.03925240e-02, -2.54433420e-02, -1.03905440e-02,       -2.00317870e-02,  9.35516300e-02, -1.25398360e-02,  2.14342160e-02,       -2.06985230e-02,  1.67553310e-02, -4.84288450e-02, -5.15675300e-02,       -1.21420820e-02,  7.48475600e-04, -2.12934670e-02, -1.69793130e-02,       -2.51965530e-03,  6.76740300e-03, -2.25207950e-02,  4.35570100e-02,        2.09687100e-04, -2.29196720e-02,  7.18679000e-03, -6.64589700e-02,       -2.66536370e-04,  1.23224620e-02, -2.73203960e-02,  5.86651820e-02,       -2.35326220e-02, -1.44412610e-02,  1.09296470e-03, -3.15908400e-02,       -1.97403880e-02, -5.55518830e-03, -2.99533480e-02,  3.19716400e-02,        1.00455220e-02, -1.82258260e-02, -1.56531500e-02,  6.28074260e-02,       -1.74021100e-02,  7.80431500e-03, -5.09146040e-02,  2.10228730e-02,        2.14409920e-03, -6.34366100e-02,  1.37802550e-02,  1.39

In [ ]:
#max_length = max([len(emb) for emb in df_bulas['embeddings']])
#df_bulas['embeddings'] = [np.pad(emb, (0, max_length - len(emb)), 'constant') for emb in df_bulas['embeddings']]
from keras.preprocessing.sequence import pad_sequences

In [ ]:
def generate_query(query, base, model):
  embedding_query = genai.embed_content(model = model,content = query, task_type ='RETRIEVAL_QUERY')
  #print(embedding_query)
  #print(f"Shape of df_bulas['embeddings'][0]: {df_bulas['embeddings'].shape}")
  #print(f"Shape of embedding_query['embeddings']: {len(embedding_query['embedding'])}")
  max_length = max([len(emb) for emb in df_bulas_split['embeddings']])
  df_embeddings = pad_sequences(df_bulas_split['embeddings'], maxlen=max_length, padding='post')
  scalar_product = np.dot(np.stack(df_embeddings), embedding_query["embedding"])
  index = np.argmax(scalar_product)
  return df_bulas_split.iloc[index]['texto']

In [ ]:
#query = 'how to change gears in the car'
#generate_query(query,df,model)
prompt = input("Prompt: ")
while prompt.upper().strip() != "PARAR":
  response = generate_query(prompt,df_bulas,model)
  print("Reposta: ","\n",response,"\n")
  prompt = input("Prompt: ")

Prompt: diclofenaco dose


ValueError: invalid literal for int() with base 10: '[array([ 9.45394800e-03, -4.64319800e-02, -7.72205800e-02,  1.57492440e-02,\n        9.76961100e-02,  2.03925240e-02, -2.54433420e-02, -1.03905440e-02,\n       -2.00317870e-02,  9.35516300e-02, -1.25